## **Name : Tanubrata Dey (td2201)**

An alternative to Baum-Welch for fitting the parameters of HMMs is Viterbi training. Viterbi training is similar to Baum-Welch, except that instead of calculating the probability of each transition at each time step, we run the Viterbi algorithm and count the number of transitions. In the HMM package, the viterbiTraining function runs this algorithm and works just like baumWelch: 	viterbiTraining(KittenGuess, KittenSim1000$observation) 

To compare these two algorithms, we will need a metric. There are many options, but let’s use the maximum absolute error of the transition matrix: 	max(abs(KittenHMMEstimate$hmm$transProbs-Tmat)) 
1.	Simulate 1000 observations from the kitten HMM using the modified transition and emission matrix I used in ExpectationMaximization.ipynb 
2.	Run both Baum-Welch and Viterbi training on these observations using the same initial guess as in ExpectationMaximization.ipynb. How do the maximum absolute errors compare? How do the algorithm run times compare? 
3.	Now try using an initial guess with the following transition matrix: T_guess = rbind(c(1/2,1/2,0),c(0,1/2,1/2),c(1,0,0)). How do the maximum absolute errors compare? 
4.	Try a couple other initial guesses to get a sense of how the initial guess effects the final result for these two algorithms. See if you can break Baum-Welch.

In [1]:
### R recommended

%%capture
%load_ext rpy2.ipython

In [2]:
%%capture
%%R
install.packages("HMM")
library(HMM)

**Q1. Simulate 1000 observations from the kitten HMM using the modified transition and emission matrix I used in ExpectationMaximization.ipynb** 

In [70]:
%%R
Tmat = rbind(c(0.95,0.05,0.0),c(0.0,0.9,0.1),c(1.0,0.0,0.0))
Pi = c(1.0,0.0,0.0)
Emat = rbind(c(0.1,0.9),c(0.9,0.1),c(0.5,0.5))
KittenHMM = initHMM(c("S","P","E"), c("N","Q"), startProbs=Pi, transProbs=Tmat, emissionProbs=Emat)
KittenSim_1000 = simHMM(KittenHMM,1000)
print(paste(KittenSim_1000$states,collapse=''))
print(paste(KittenSim_1000$observation,collapse=''))

[1] "SSSSPPPPPESSSSPPPPPPESSSPPPPPPPPPPPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPPPPPPPPPPPESSSPPPPPPPPPPESSSSSSSSSSSSSSPPPPPPPPPPPPPPPPPPPPPESSSSSSSSSPPPPPPPPPPPESSSSSSSSSSSSSSSSSSSPPPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPPPPPPPPPPPPPPPPPPESSSPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPESSSSSSSSSSSSSSSPPPPPPPPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPPPESSPPPPPPPPPPPPPPESSSSSPPPPESSSSSSPPPPPESSSSSSSSSSSSSSSSPPESSSSSSPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPESSSSSSSSSSPPPPPPPPPPPPPPPESSSSSSSSSSSSSSSSSSSPPPPPPPPPPESSSSSSSSSPPPESSSSSSSSSSSSSSSSSSSSSPPPPPPPPPPPPPESSSSSSSSSPPPPPPPPPPPPPPPPPPPPPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPESSSSPPPPPESSSSSSSSSSSSSSSSSPPESSSSSSSSSSSSSSSSSSSSSSSPPPESSSSSSSSSSSSSSSSSSSPPPPPPPPPPPPPPPPPPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPPPPPPPPPPPPPPESSSSSSSSSSPESSSSSPPPPPPPPPPPPPPESSSSSSSSSSSSSSSSSSSSSSSSPPESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSPPPPPPESSSSSSSSSSSSSSSSSSSP

***

**Q2. Run both Baum-Welch and Viterbi training on these observations using the same initial guess as in ExpectationMaximization.ipynb. How do the maximum absolute errors compare? How do the algorithm run times compare?** 

In [60]:
%%R
T_guess = rbind(c(0.9,0.1,0.0),c(0.0,0.8,0.2),c(1.0,0.0,0.0))
E_guess = rbind(c(0.1,0.9),c(0.9,0.1),c(0.5,0.5))
KittenGuess = initHMM(c("S","P","E"), c("N","Q"), transProbs=T_guess, emissionProbs=E_guess)
#KittenSim_1000 = simHMM(KittenHMM,1000)


In [61]:
%%R
KittenHMMEstimate = baumWelch(KittenGuess, KittenSim_1000$observation) #Baum-Welch
print(KittenHMMEstimate$hmm$transProbs)
print(KittenHMMEstimate$hmm$emissionProbs)

    to
from         S         P         E
   S 0.9537048 0.0462952 0.0000000
   P 0.0000000 0.8777315 0.1222685
   E 1.0000000 0.0000000 0.0000000
      symbols
states          N          Q
     S 0.07461646 0.92538354
     P 0.92028413 0.07971587
     E 0.10487812 0.89512188


In [62]:
%%R
KittenHMMEstimate_Viterbi = viterbiTraining(KittenGuess, KittenSim_1000$observation) #Viterbi
print(KittenHMMEstimate_Viterbi$hmm$transProbs)
print(KittenHMMEstimate_Viterbi$hmm$emissionProbs)

    to
from         S          P         E
   S 0.9565217 0.04347826 0.0000000
   P 0.0000000 0.86147186 0.1385281
   E 1.0000000 0.00000000 0.0000000
      symbols
states          N          Q
     S 0.06648575 0.93351425
     P 0.94805195 0.05194805
     E 1.00000000 0.00000000


In [63]:
%%R
#maximum absolute errors
print(max(abs(KittenHMMEstimate$hmm$transProbs - Tmat))) #MAE of Baum_Welch
print(max(abs(KittenHMMEstimate_Viterbi$hmm$transProbs - Tmat))) #MAE of Viterbi


[1] 0.02226848
[1] 0.03852814


The maximum absolute error for Baum-Welch algorithm is ~0.022 and for Viterbi algorithm, its ~0.038 and it pretty much looks both of them performed well but kind of looks like Baum-Welch gave ~0.01 less error compared to Viterbi which is acceptable. Now comparing how much time each took, Baum-Welch almost took 14 seconds whereas Viterbi took only 4 seconds which is pretty low meaning Viterbi worked much faster compared to Baum-Welch.

***

**3. Now try using an initial guess with the following transition matrix: T_guess = rbind(c(1/2,1/2,0),c(0,1/2,1/2),c(1,0,0)). How do the maximum absolute errors compare?**

In [64]:
%%R
T_guess =  rbind(c(1/2, 1/2, 0), c(0, 1/2, 1/2), c(1, 0, 0))
E_guess = rbind(c(0.1,0.9),c(0.9,0.1),c(0.5,0.5))
KittenGuess = initHMM(c("S","P","E"), c("N","Q"), transProbs=T_guess, emissionProbs=E_guess)
#KittenSim_1000 = simHMM(KittenHMM,1000)


In [65]:
%%R
KittenHMMEstimate = baumWelch(KittenGuess, KittenSim_1000$observation) #Baum-Welch
print(KittenHMMEstimate$hmm$transProbs)
print(KittenHMMEstimate$hmm$emissionProbs)

    to
from         S          P         E
   S 0.9530875 0.04691247 0.0000000
   P 0.0000000 0.87837441 0.1216256
   E 1.0000000 0.00000000 0.0000000
      symbols
states          N          Q
     S 0.07408977 0.92591023
     P 0.91960554 0.08039446
     E 0.03200341 0.96799659


In [66]:
%%R
KittenHMMEstimate_Viterbi = viterbiTraining(KittenGuess, KittenSim_1000$observation) #Viterbi
print(KittenHMMEstimate_Viterbi$hmm$transProbs)
print(KittenHMMEstimate_Viterbi$hmm$emissionProbs)

    to
from         S         P         E
   S 0.8654434 0.1345566 0.0000000
   P 0.0000000 0.6575875 0.3424125
   E 1.0000000 0.0000000 0.0000000
      symbols
states         N          Q
     S 0.0000000 1.00000000
     P 0.9805447 0.01945525
     E 0.5454545 0.45454545


In [68]:
%%R
#maximum absolute errors 
print(max(abs(KittenHMMEstimate$hmm$transProbs - Tmat))) #MAE of Baum_Welch
print(max(abs(KittenHMMEstimate_Viterbi$hmm$transProbs - Tmat))) #MAE of Viterbi


[1] 0.02162559
[1] 0.2424125


By changing initial T_guess, the maximum absolute error for Baum-Welch algorithm is ~0.021 and for Viterbi algorithm its ~0.242 and comparing the values it looks like Baum-Welch performed much better whereas Viterbi was not that much efficient this time. Based on the results it looks Baum-Welch gives more accurate output, though it takes more time.

***

**4. Try a couple other initial guesses to get a sense of how the initial guess effects the final result for these two algorithms. See if you can break Baum-Welch.**

In [71]:
%%R
T_guess = rbind(c(0.5,0.9,0.3),c(0.2,0.1,0.9),c(1.0,0.0,0.0))
E_guess = rbind(c(0.1,0.9),c(0.9,0.1),c(0.5,0.5))
KittenGuess = initHMM(c("S","P","E"), c("N","Q"), transProbs=T_guess, emissionProbs=E_guess)
#KittenSim_1000 = simHMM(KittenHMM,1000)

KittenHMMEstimate = baumWelch(KittenGuess, KittenSim_1000$observation) #Baum-Welch
print(KittenHMMEstimate$hmm$transProbs)
print(KittenHMMEstimate$hmm$emissionProbs)
print(max(abs(KittenHMMEstimate$hmm$transProbs - Tmat))) #MAE of Baum_Welch

    to
from          S          P          E
   S 0.64852757 0.07444616 0.27702627
   P 0.02800166 0.90788697 0.06411137
   E 1.00000000 0.00000000 0.00000000
      symbols
states          N          Q
     S 0.11294898 0.88705102
     P 0.91015871 0.08984129
     E 0.06987257 0.93012743
[1] 0.3014724


In [84]:
%%R
T_guess = rbind(c(0.9,0.8,0.5),c(0.2,0.9,0.5),c(1.0,0.9,0.0))
E_guess = rbind(c(0.1,0.9),c(0.9,0.1),c(0.5,0.5))
KittenGuess = initHMM(c("S","P","E"), c("N","Q"), transProbs=T_guess, emissionProbs=E_guess)
#KittenSim_1000 = simHMM(KittenHMM,1000)

KittenHMMEstimate = baumWelch(KittenGuess, KittenSim_1000$observation) #Baum-Welch
print(KittenHMMEstimate$hmm$transProbs)
print(KittenHMMEstimate$hmm$emissionProbs)
print(max(abs(KittenHMMEstimate$hmm$transProbs - Tmat))) #MAE of Baum_Welch

    to
from         S          P         E
   S 0.8105639 0.01681941 0.1726167
   P 0.0222462 0.88169890 0.0960549
   E 0.7229649 0.27703510 0.0000000
      symbols
states          N          Q
     S 0.09565806 0.90434194
     P 0.93176565 0.06823435
     E 0.16558278 0.83441722
[1] 0.2770351


In [86]:
%%R
T_guess = rbind(c(0.1,0.0,0.1),c(0.1,0.1,0.0),c(0.1,0.0,0.1))
E_guess = rbind(c(0.1,0.9),c(0.9,0.1),c(0.5,0.5))
KittenGuess = initHMM(c("S","P","E"), c("N","Q"), transProbs=T_guess, emissionProbs=E_guess)
#KittenSim_1000 = simHMM(KittenHMM,1000)

KittenHMMEstimate = baumWelch(KittenGuess, KittenSim_1000$observation) #Baum-Welch
print(KittenHMMEstimate$hmm$transProbs)
print(KittenHMMEstimate$hmm$emissionProbs)
print(max(abs(KittenHMMEstimate$hmm$transProbs - Tmat))) #MAE of Baum_Welch

R[write to console]: Error in if (temp > -Inf) { : missing value where TRUE/FALSE needed




Error in if (temp > -Inf) { : missing value where TRUE/FALSE needed


RInterpreterError: ignored

Tmat = rbind(c(0.95,0.05,0.0),c(0.0,0.9,0.1),c(1.0,0.0,0.0))

T_guess = rbind(c(0.1,0.0,0.1),c(0.1,0.1,0.0),c(0.1,0.0,0.1))

If you compare the two matrices, the first one that's given that is Tmat has value in the middle of first vector whereas in the T_guess I changed that value to 0 meaning the initial guess probability is 0 whereas the given actual have value leading Baum-Welch to crash. 

***